In [1]:
from IPython.display import Markdown

In [2]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

In [3]:
from pydantic import BaseModel, Json
from typing import Deque, List, Optional, Tuple, Dict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os
import json
import requests
from bs4 import BeautifulSoup

In [4]:
load_dotenv()

True

In [5]:
ollama_url = os.getenv("OLLAMA_BASE_URL")
llm_model = os.getenv("LLM_MODEL")
llm_model

'qwen3:8b-q4_K_M'

In [6]:
from langgraph.prebuilt import create_react_agent
from stripe_agent_toolkit.langchain.toolkit import StripeAgentToolkit

# Tools

In [10]:
def get_text_content(url: str) -> str:
    """
    Fetches the HTML content of a given URL.
    """
    try: 
        print(f"Fetching: {url}")
        response = requests.get(url, timeout=10)
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        for tag in soup(["script", "style", "footer", "nav", "aside", "noscript"]):
            tag.decompose()
        article = soup.find("article")
        if article:
            out_text = article.get_text(" ", strip=True)
        else: 
            out_text = soup.get_text(" ", strip=True)
        return out_text
    else:
        return ""

In [ ]:
content = get_html_content('https://www.cbsnews.com/news/federal-reserve-meeting-today-fed-fomc-interest-rate-trump-powell')
content

In [32]:
for tag in content(["script", "style", "footer", "nav", "aside", "noscript"]):
    tag.decompose()
out_text = content.get_text(" ", strip=True)

In [33]:
out_text

'Federal Reserve holds its benchmark rate steady at today\'s FOMC meeting - CBS News Watch CBS News MoneyWatch Federal Reserve holds its benchmark rate steady at today\'s FOMC meeting By Aimee  Picchi Aimee  Picchi Associate Managing Editor, MoneyWatch Aimee Picchi is the associate managing editor for CBS MoneyWatch, where she covers business and personal finance. She previously worked at Bloomberg News and has written for national news outlets including USA Today and Consumer Reports. Read Full Bio Aimee  Picchi Edited By Alain  Sherter , Alain  Sherter Senior Managing Editor, MoneyWatch Alain Sherter is a senior managing editor with CBS News. He covers business, economics, money and workplace issues for CBS MoneyWatch. Read Full Bio Alain  Sherter , Anne Marie D. Lee Anne Marie D. Lee Editor, MoneyWatch Anne Marie D. Lee is an editor for CBS MoneyWatch. She writes about topics including personal finance, the workplace, travel and social media. Read Full Bio Anne Marie D. Lee Updated 

In [36]:
article = content.find("article") 
article.get_text(" ", strip=True)

'MoneyWatch Federal Reserve holds its benchmark rate steady at today\'s FOMC meeting By Aimee  Picchi Aimee  Picchi Associate Managing Editor, MoneyWatch Aimee Picchi is the associate managing editor for CBS MoneyWatch, where she covers business and personal finance. She previously worked at Bloomberg News and has written for national news outlets including USA Today and Consumer Reports. Read Full Bio Aimee  Picchi Edited By Alain  Sherter , Alain  Sherter Senior Managing Editor, MoneyWatch Alain Sherter is a senior managing editor with CBS News. He covers business, economics, money and workplace issues for CBS MoneyWatch. Read Full Bio Alain  Sherter , Anne Marie D. Lee Anne Marie D. Lee Editor, MoneyWatch Anne Marie D. Lee is an editor for CBS MoneyWatch. She writes about topics including personal finance, the workplace, travel and social media. Read Full Bio Anne Marie D. Lee Updated on:  July 30, 2025 / 7:31 PM EDT / CBS News Interest rates remain unchanged: What to know Powell ci

In [11]:
def fetch_news_api(country: str):
    """
    Tool that fetches news articles from News API
    """
    load_dotenv()
    news_api_key = os.getenv("NEWS_API_KEY")
    match country:
        case "US":
            country = "us"
        case "Brazil":
            country = "br"
        case "Japan":
            country = "jp"
    url =  f"https://api.webz.io/newsApiLite?token={news_api_key}&q=published%3A%3Enow-24h%20site_category%3Atop_news_{country}%20performance_score%3A%3E0%20country%3A{country}"
    
    response = requests.get(url, timeout=10)
    print(response)
    if response.status_code == 200:
        data = response.json()
        articles = data["posts"]
        output = []
        for article in articles:
            source_url = article["url"]
            title = article["title"]
            content = get_text_content(source_url)
            output.append({
                "title": title,
                "content": content,            })
        return {"trending_news": output}
    else:
        return {"error": f"Error fetching news: {response.status_code}"}

In [12]:
output = fetch_news_api("US")

<Response [200]>
Fetching: https://www.cbsnews.com/news/federal-reserve-meeting-today-fed-fomc-interest-rate-trump-powell/
Fetching: https://www.usatoday.com/story/news/nation/2025/07/28/tarantula-mating-season-is-coming-brace-for-hordes-of-them/85276029007/
Fetching: https://www.cbsnews.com/pictures/son-of-sam-serial-killer-david-berkowitz-victims-and-timeline/
Fetching: https://www.cnbc.com/2025/07/30/fed-leaves-interest-rates-unchanged-as-expected.html
Fetching: https://www.foxnews.com/world/arab-nations-first-time-pressure-hamas-disarm-relinquish-power-gaza
Fetching: https://www.washingtonpost.com/politics/2025/07/30/kamala-harris-governor-president-2028/
Error fetching https://www.washingtonpost.com/politics/2025/07/30/kamala-harris-governor-president-2028/: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)
Fetching: https://www.cnn.com/audio/podcasts/all-there-is-with-anderson-cooper
Fetching: https://www.cbsnews.com/news/hawley-democ

In [ ]:
Markdown(graph_state['final_output'])

# Nodes

In [33]:
news_keyword_system_prompt = """You are a professional journalist and writer. Your goal is to find the current important news around the world. Your tasks consist of two parts. 
1. Rank the article by the importance and pick top 3.  
2. List keywords that capture the essense of each news. Don't add more than 5 keywords.
Here is the list of current trending news.
{news}
Format your response in JSON format. 
Example Output:
    [{{
      "Title": "Magnitude 8.7 earthquake hits Russia’s Kamchatka, tsunami warnings issued",
      "Keywords": ["earthquake", "tsunami", "Kamchatka", "Japan"]
    }},
    {{
      "Title": "Gaza crisis worsens with death toll exceeding 60,000",
      "Keywords": ["Gaza", "Israel-Hamas war", "famine", "humanitarian crisis"]
    }}
    ...
    ]

"""

In [ ]:
# 1. Rank the given trending news by the importance and pick top 3~5.  
# 2. Generate an engaging and creative article based on the news you picked.

In [39]:
writer_system_prompt = """You are a professional journalist and writer. Your goal is to determine the current important news from the given trending news, synthesize information, get insights, and write a newsletter. Don't just explain or summarize the news. 
Instead of providing a segmented or itemized summary, weave the content into a continuous narrative with creative transitions, and write in an engaging storytelling tone. 
Ensure the story has a clear beginning, middle, and end, and that each part flows naturally into the next, making it feel like one unified piece rather than multiple separate news items.

Here is the list of current trending news.
{news}

Format your response in JSON format. 
Expected Format:
    {{
      "Article": "Your writing here",
    }}

"""

In [41]:
class AgentState(BaseModel):
    interests: Optional[List[str]] = None
    country: str = "US"
    agent_type: str = "curation"
    # queries: Optional[List[str]] = None
    trending_news: Optional[List[Dict[str, str]]] = None
    sources_used: Optional[List[str]] = None
    is_subscribed: Optional[bool] = None
    payment_status: Optional[str] = None
    final_output: Optional[str] = None

def list_trending_news(state: AgentState) -> AgentState:
    """
    Langgraph node that finds trending news 
    """
    response = fetch_news_api(state.country)
    state.trending_news = response["trending_news"]
    return state

    
def generate_newsletter(state: AgentState) -> AgentState:
    """
    Langgraph node that generates newsletter
    """
    ollama_url = os.getenv("OLLAMA_BASE_URL")
    llm_model = os.getenv("LLM_MODEL_LIGHT")
    chat_ollama = ChatOllama(
            base_url=ollama_url, 
            model=llm_model, 
            # format="json"
        )
    news_str = "\n".join(f"Title: {news['title']} Content: {news['content']})" for news in state.trending_news)
    user_message = [SystemMessage(content=writer_system_prompt.format(news=news_str)), HumanMessage(content="Write a newsletter using trending news")]
    result = chat_ollama.invoke(user_message)
    state.final_output = result.content
    return state
    

builder = StateGraph(AgentState)
builder.add_node("list_trending_news", list_trending_news)
builder.add_node("generate_newsletter", generate_newsletter)

builder.add_edge(START, "list_trending_news")
builder.add_edge("list_trending_news", "generate_newsletter")
builder.add_edge("generate_newsletter", END)
graph = builder.compile()

In [42]:
US_newsarticle = graph.invoke({"country": "US"})

KeyboardInterrupt: 

In [ ]:
#gemma3n
Markdown(US_newsarticle['final_output'])

In [ ]:
graph_state = graph.invoke({})

In [ ]:
#qwen3
Markdown(US_newsarticle['final_output'])

In [9]:
#gemma3n
Markdown(graph_state['final_output'])

Here's a summary of the trending news, synthesized from the provided headlines:

**Key Themes:**

* **Crime & Justice:** The "Son of Sam" serial killer case continues to be a prominent story, highlighting a dark chapter in American history.  A disturbing incident in Virginia involving a city council member being set on fire is also making headlines, with an arrest made.  Furthermore, Democrats are pushing to release files related to Jeffrey Epstein using a rarely used legal mechanism.
* **Economy & Finance:** The Federal Reserve has decided to hold its benchmark interest rate steady at its latest FOMC meeting, defying calls from some (including Donald Trump) for more aggressive cuts. This is a significant economic development.
* **Politics & Legislation:**  A bipartisan effort in Congress is gaining momentum to ban members of Congress from trading stocks.  Arab nations are taking a previously unprecedented stance by calling for Hamas disarmament and condemning the October 7th attacks.
* **Business & Workplace:**  Forbes has released its list of "America's Best Employers for Women 2025," highlighting companies with strong workplace cultures and opportunities for women.



**In short:**  The news cycle is dominated by a mix of crime, economic policy, political maneuvering, and corporate recognition.  The Federal Reserve's decision on interest rates is a major economic event, while the "Son of Sam" case and the Virginia attack are capturing public attention due to their disturbing nature.  Bipartisan efforts in Congress and a shift in Arab nations' approach to Hamas are also noteworthy developments.



**Note:**  The inclusion of "Video - The Washington Post" suggests that video content from the Washington Post is also a significant part of the current news landscape, though the specific topic isn't detailed in the headline.





In [27]:
#qwen
Markdown(graph_state['final_output'])

<think>
Okay, the user wants me to synthesize and summarize trending news from the provided sources. Let me start by going through each title and the corresponding article links. Wait, the links are all from different sources like CBS News, Forbes, Fox News, etc. The user mentioned "around the world," so I need to make sure I cover both U.S. and international topics.

First, the "Son of Sam" article is about a historical serial killer, David Berkowitz. That's a U.S. crime story from the past. The Federal Reserve rate decisions are repeated in two sources, so I should note that the Fed kept rates steady, defying Trump's demands. Then there's Forbes' list of best employers for women in 2025, which is a positive business development. 

The Arab nations backing Hamas disarmament and condemning the Oct 7 massacre is an international news item. The GOP Sen. Josh Hawley and Democrats advancing the stock trading ban is a U.S. political development. The video from The Washington Post is vague, but maybe it's a recent event. The Senate Democrats seeking Epstein files via the Rule of Five is another U.S. legal issue. The Virginia council member attack is a local U.S. incident. 

I need to group these by themes: U.S. politics, economy, crime, international relations, and business. Also, check for any duplicates, like the two Fed rate articles. Make sure to highlight the key points from each source without getting too detailed. The user probably wants a concise summary that captures the most important trends. Let me structure it with clear sections and ensure all the main topics are covered.
</think>

Here’s a synthesized summary of trending global news and key developments from the provided sources:

---

### **U.S. Politics & Economy**  
1. **Federal Reserve Rate Decision**: The U.S. Federal Reserve kept its benchmark interest rates unchanged at its July 2025 FOMC meeting, defying President Trump’s calls for aggressive rate cuts. This decision reflects ongoing economic caution amid inflation concerns.  
2. **Stock Trading Ban**: Senate Democrats and GOP Sen. Josh Hawley advanced a congressional ban on stock trading by lawmakers, aiming to reduce conflicts of interest.  
3. **Jeffrey Epstein Files**: Senate Democrats are leveraging the rarely used "Rule of Five" law to demand the release of sealed files related to Jeffrey Epstein’s case, seeking transparency on his alleged crimes.  

---

### **Crime & Social Issues**  
1. **"Son of Sam" Legacy**: The 1970s serial killer David Berkowitz, known as the "Son of Sam," remains a historical case of urban violence, with his crimes still studied for their psychological and societal impact.  
2. **Violent Attack in Virginia**: A city council member in Virginia was set on fire with gasoline at his workplace, and the suspect was arrested, highlighting local tensions and safety concerns.  

---

### **International News**  
1. **Arab Nations Condemn Hamas**: For the first time, Arab nations publicly condemned the October 7 Hamas massacre and called for Hamas to disarm and relinquish power in Gaza, signaling a shift in regional diplomacy.  
2. **Global Disarmament Push**: The Arab stance reflects broader international pressure on Hamas to end violence and pave the way for potential ceasefire negotiations.  

---

### **Business & Workforce Trends**  
1. **Best Employers for Women 2025**: Forbes highlighted companies recognized for fostering inclusive workplaces and advancing gender equity, reflecting a growing focus on diversity in corporate America.  

---

### **Media & Technology**  
- **The Washington Post Video**: A recent video from The Washington Post (source not specified) likely covers breaking news or investigative reporting, though details remain unclear.  

---

### **Key Themes**  
- **Economic Policy**: The Fed’s rate decision underscores the tension between inflation control and growth concerns.  
- **Global Diplomacy**: Arab nations’ unified stance on Hamas marks a pivotal moment in Middle East politics.  
- **Social Justice**: Efforts to hold power accountable (e.g., Epstein files, stock trading ban) and address violence (e.g., Virginia attack) highlight ongoing societal challenges.  

This mix of domestic and international news reflects a complex landscape of economic, political, and social dynamics shaping the world in 2025.

In [11]:
stripe = graph.invoke({"topic": "Stripe"})

In [12]:
from IPython.display import Markdown
Markdown(stripe['final_output'])

<think>
Okay, the user asked, "Hello, what is Stripe?" So I need to explain what Stripe is. Let me start by recalling what I know about Stripe. It's a payment processing company, right? They help businesses handle payments online. But I should make sure I get the details right.

First, I should mention that Stripe is a financial technology company. They provide tools for developers to integrate payment systems into their websites and apps. That's a key point. But wait, maybe I should clarify that they're based in the US and have offices in other countries too. Also, their main focus is on enabling businesses to accept payments globally.

Wait, the user might not know what payment processing is. Maybe I should explain that in simple terms. Like, when a customer buys something online, Stripe helps the business get the money from the customer. But I should avoid jargon. Maybe use an example, like an e-commerce site using Stripe to process credit card payments.

Also, Stripe offers different services. There's the payment processing, but they also have tools for subscriptions, invoices, and even fraud detection. Oh, and they have a platform where developers can build their own payment solutions. That's part of their API.

I should mention that Stripe is used by many companies, both large and small. Maybe name a few examples, like Uber or Spotify, but I need to check if that's accurate. Wait, I think they do work with some big names. But I shouldn't list all of them unless I'm sure.

Another point is that Stripe handles the complexities of different payment methods and currencies. So businesses can accept payments in various currencies without dealing with the technicalities themselves. That's a big advantage.

Also, security is important. Stripe complies with PCI standards, so they handle the sensitive payment data, which is a plus for businesses. They might also offer tools for managing refunds, disputes, and customer accounts.

Wait, the user might be a business owner looking to set up online payments. So explaining the benefits would be helpful. But since they asked "what is Stripe," the focus should be on the definition and overview, not the benefits.

I should structure the answer clearly: start with a definition, then explain what they do, their services, and maybe their role in the market. Also, mention that they're a company that provides APIs and tools for developers. Maybe include their location and when they were founded, but I'm not sure about the exact year. Wait, Stripe was founded in 2011, I think. Let me confirm that. Yes, 2011 in San Francisco.

Wait, but maybe the user doesn't need the exact founding date. Maybe just mention that they're a well-known company. Also, their main product is the payment processing platform, which is used by millions of businesses.

I should also note that Stripe is not just for e-commerce. They support various industries, including SaaS, marketplaces, and more. Their tools can be integrated into different types of websites and apps.

Hmm, I need to make sure the explanation is clear and concise. Avoid technical terms where possible. Let me put this all together in a coherent way.
</think>

Stripe is a financial technology company that provides tools and services to help businesses accept, process, and manage online payments. Founded in 2011 in San Francisco, Stripe is known for its developer-friendly platform that enables businesses to integrate payment systems into their websites, apps, and services with ease.

### Key Features of Stripe:
1. **Payment Processing**: Stripe allows businesses to accept payments via credit cards, debit cards, and other digital payment methods. It handles the technical complexities of payment gateways, fraud detection, and currency conversion.
2. **Global Reach**: It supports transactions in multiple currencies and operates in numerous countries, making it ideal for international businesses.
3. **APIs for Developers**: Stripe offers a robust API (Application Programming Interface) that developers can use to customize payment solutions, such as subscriptions, invoicing, and recurring payments.
4. **Security**: Stripe complies with strict security standards (e.g., PCI DSS) to protect sensitive payment data, reducing the burden on businesses to manage security themselves.
5. **Additional Services**: Beyond payments, Stripe provides tools for managing refunds, disputes, customer accounts, and analytics.

### Use Cases:
- **E-commerce**: Online retailers use Stripe to process payments on their websites.
- **SaaS (Software as a Service)**: Companies charge recurring subscriptions using Stripe's tools.
- **Marketplaces**: Platforms like Uber or Airbnb use Stripe to handle payments between users and service providers.

### Why Stripe Stands Out:
Stripe is popular for its simplicity, scalability, and focus on developer experience. It caters to businesses of all sizes, from startups to large enterprises, and is used by well-known companies like Spotify, Netflix, and Shopify.

In short, Stripe acts as a "payment engine" for businesses, enabling them to focus on their core operations while Stripe handles the complexities of payment processing.

In [8]:
class AgentState(BaseModel):
    agent_type: str = "curation"
    topic: str = ""
    subscribed: bool = False
    billing_result: str = ""
    final_output: str = ""

def generate_query(state: AgentState) -> AgentState:
    """
    Langgraph node that generates query
    """
    ollama_url = os.getenv("OLLAMA_BASE_URL")
    llm_model = os.getenv("LLM_MODEL")
    chat_ollama = ChatOllama(
            base_url=ollama_url, 
            model=llm_model, 
        )
    user_message = HumanMessage(content=f"Hello, what is the current trending news in {state.topic}?")
    result = chat_ollama.invoke([user_message])
    state.final_output = result.content
    return state

def charge_subscription_fee(state: AgentState) -> AgentState:
    """
    Langgraph node that charges monthly subscription fee, $1/month
    """
    openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
    
    ollama_url = os.getenv("OLLAMA_BASE_URL")
    llm_model = os.getenv("LLM_MODEL")
    chat_ollama = ChatOllama(
            base_url=ollama_url, 
            model=llm_model, 
        )
    stripe_agent_toolkit = StripeAgentToolkit(
        secret_key=os.getenv("STRIPE_SECRET_KEY"),
        configuration={
            "actions": {
                "payment_links": {
                    "create": True,
                },
                "products": {
                    "create": True,
                    },
                "prices": {
                    "create": True,
                    },
                }
            },
        )
    tools = stripe_agent_toolkit.get_tools()
    langgraph_agent_executor = create_react_agent(chat_ollama, tools)
    input_state = {
    "messages": """
        Create a payment link for a new product called 'test' with a price
        of $1.
    """,
    }
    state.billing_result = langgraph_agent_executor.invoke(input_state)
    state.subscribed = True
    state.topic = "Stripe"
    return state
    
    

builder = StateGraph(AgentState)
builder.add_node("charge_subscription_fee", charge_subscription_fee)
builder.add_node("generate_query", generate_query)

builder.add_edge(START, "charge_subscription_fee")
# builder.add_edge("charge_subscription_fee", "generate_query")
# builder.add_edge("generate_query", END)
builder.add_edge("charge_subscription_fee", END)
graph = builder.compile()

In [9]:
graph.invoke({"topic": "fake here"})

/home/yudaih/.cache/pypoetry/virtualenvs/newsletter-langgraph-xa5oPsqH-py3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value={'messages': [HumanMessag...'total_tokens': 3173})]}, input_type=dict])
  return self.__pydantic_serializer__.to_python(


{'agent_type': 'curation',
 'topic': 'Stripe',
 'subscribed': True,
 'billing_result': {'messages': [HumanMessage(content="\n        Create a payment link for a new product called 'test' with a price\n        of $1.\n    ", additional_kwargs={}, response_metadata={}, id='d8fb3ba0-ee19-4d93-8c1a-09db7a7daf82'),
   AIMessage(content='<think>\nOkay, let\'s see. The user wants to create a payment link for a new product called \'test\' priced at $1. Hmm, I need to figure out which functions to use here.\n\nFirst, the product needs to be created. The \'create_product\' function is for that. The name is \'test\', and maybe a description, but the user didn\'t specify one. So I\'ll just provide the name.\n\nNext, creating a price for the product. The \'create_price\' function requires the product ID, unit amount, and currency. The unit amount is $1, which is 100 cents. Currency is probably USD unless specified otherwise. But the user didn\'t mention the currency, so I should assume USD.\n\nWait

In [ ]:
from IPython.display import Markdown
Markd